In [1]:
import pandas as pd

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import Lambda

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

from tqdm import tqdm

# all the functions from helpers.py
from helpers_scenario2 import *

import pandas as pd

from joblib import Parallel, delayed, parallel_backend
import multiprocessing
from multiprocessing import Value
from tqdm.auto import tqdm

# all the functions from helpers.py
from helpers_scenario2 import *

from keras.layers import Dropout, LSTM
from keras.regularizers import l1, l2
from keras.layers import Activation
from sklearn.preprocessing import StandardScaler
from kerastuner.tuners import RandomSearch

from keras.layers import MaxPooling1D

import json
from keras.layers import Dropout, LSTM
from keras.layers import Activation
from sklearn.preprocessing import StandardScaler
from kerastuner.tuners import RandomSearch

from keras.layers import MaxPooling1D


C:\Users\xochipilli\AppData\Local\Temp\ipykernel_11972\1464098886.py:35: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [2]:
scenario_folder = "../data/preprocessed/cleaned_and_prepro_improved/scenario_2/"
fold = 0

phys_folder, ann_folder = create_folder_structure(scenario_folder, fold)
annotations_folder = '../data/raw/scenario_2/fold_0/train/annotations/'
physiology_folder = "../data/preprocessed/cleaned_and_prepro_improved/scenario_2/fold_0/train/physiology/" 
zipped_files = zip_csv_files(annotations_folder, physiology_folder)

subjects, videos = get_subs_vids(physiology_folder)

num_cpu_cores = multiprocessing.cpu_count()

def process_files(annotation_file, physiology_file,):
    df_annotations = pd.read_csv(annotation_file)
    df_physiology = pd.read_csv(physiology_file)
    
    print(physiology_file)
    X, y, numeric_column_indices = preprocess(df_physiology, df_annotations, predictions_cols=['arousal','valence'], aggregate=None, window=[-10000, 10000], partition_window = 1)
    # print(X.shape, y.shape)
    
    save_files(X, y, annotation_file, phys_folder, ann_folder)
    
    return None

# zipped_files = zipped_files[115:]

## Process the files using the context manager
with parallel_backend('loky', n_jobs=num_cpu_cores // 3):
    with tqdm_joblib(tqdm(total=len(zipped_files), desc="Files", leave=False)) as progress_bar:
        results = Parallel()(
            (delayed(process_files)(ann_file, phys_file) for ann_file, phys_file in zipped_files)
        )

#for ann_file, phys_file in zipped_files:
#    process_files(ann_file, phys_file,)
    


Files:   0%|          | 0/192 [00:00<?, ?it/s]

In [ ]:
scenario_folder = "../data/preprocessed/cleaned_and_prepro_improved/scenario_2/"
fold = 0

phys_folder, ann_folder = create_folder_structure(scenario_folder, fold)

In [ ]:
print(phys_folder)

In [ ]:
annotations_folder = '../data/raw/scenario_2/fold_0/train/annotations/'
physiology_folder = "../data/preprocessed/cleaned_and_prepro_improved/scenario_2/fold_0/train/physiology/" 
zipped_files = zip_csv_files(annotations_folder, physiology_folder)

subjects, videos = get_subs_vids(physiology_folder)

# Comment this lineas
videos = [0]
subjects = [0,1,11]

splits = split_subjects_train_test(subjects, 3)

In [ ]:
print(subjects)

In [ ]:

def create_cnn_lstm_model(input_shape, lstm_units=64, dropout_rate=0.4, kernel_regularizer_l1=0.0032):
    input_signal = Input(shape=input_shape)

    x = Conv1D(16, 5, padding='same')(input_signal)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(dropout_rate)(x)

    x = MaxPooling1D(pool_size=2)(x)

    x = LSTM(lstm_units)(x)

    x = Dense(32, activation='relu', kernel_regularizer=l1(kernel_regularizer_l1))(x)

    return Model(inputs=input_signal, outputs=x)

In [ ]:
[split for split in splits]

In [ ]:
for split in splits:
    X_train, X_test = load_and_concatenate_files(phys_folder, split, videos)
    y_train, y_test = load_and_concatenate_files(ann_folder, split, videos)

In [ ]:
train_data = []
test_data = []

In [ ]:
file_path = os.path.join(phys_folder, f"sub_0_vid_9.npy")

In [ ]:
if os.path.exists(file_path):
    data = np.load(file_path)
    train_data.append(data)

In [ ]:
train_data[2].shape

In [ ]:
train_data_cc = np.concatenate(train_data, axis=1) if train_data else None


In [ ]:
for train_test, subs in train_test_split.items():
    if type(vid) == 'list':
        for v in vid:
            for sub in subs:
                file_path = os.path.join(base_path, f"sub_{sub}_vid_{v}.npy")
                if os.path.exists(file_path):
                    data = np.load(file_path)
                    if train_test == "train":
                        train_data.append(data)
                    else:
                        test_data.append(data)

In [ ]:


    for train_test, subs in train_test_split.items():
        if type(vid) == 'list':
            for v in vid:
                for sub in subs:
                    file_path = os.path.join(base_path, f"sub_{sub}_vid_{v}.npy")
                    if os.path.exists(file_path):
                        data = np.load(file_path)
                        if train_test == "train":
                            train_data.append(data)
                        else:
                            test_data.append(data)

        else:
            for sub in subs:
                file_path = os.path.join(base_path, f"sub_{sub}_vid_{vid}.npy")
                if os.path.exists(file_path):
                    data = np.load(file_path)
                    if train_test == "train":
                        train_data.append(data)
                    else:
                        test_data.append(data)

    train_data = np.concatenate(train_data) if train_data else None
    test_data = np.concatenate(test_data) if test_data else None

In [ ]:
print(y_test)

In [ ]:
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

In [ ]:
from concurrent.futures import ProcessPoolExecutor, as_completed, ThreadPoolExecutor
import threading
import multiprocessing
from tqdm import tqdm
import itertools
import json

# Initialize an empty dictionary to store the fold results
fold_results = {}

# Loop through the splits
for split_index, split in enumerate(splits):

        X_train, X_test, y_train, y_test, numeric_column_indices, categorical_column_indices = preprocess(
            df_physiology_video.copy(), df_annotations_video.copy(), split=split, predictions_cols=['arousal', 'valence'], aggregate=None,
            window_duration=10000, resample_rate=100)

        print("X_test shape:", X_test.shape)
        print("y_test shape:", y_test.shape)
                
        # Extract arousal and valence values from y_train and y_test
        y_arousal_train = y_train[:, 0]
        y_valence_train = y_train[:, 1]
        y_arousal_test = y_test[:, 0]
        y_valence_test = y_test[:, 1]

        # Normalize the input data
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        input_shape = (X_train.shape[1], X_train.shape[2])

        # Define a Keras wrapper for the model
        def model_builder(hp):
            lstm_units = hp.Int('lstm_units', min_value=32, max_value=128, step=32)
            dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)
            kernel_regularizer_l1 = hp.Choice('kernel_regularizer_l1', values=[0.001, 0.01, 0.1])

            model = create_cnn_lstm_model(input_shape, lstm_units, dropout_rate, kernel_regularizer_l1)

            valence_output = Dense(1, activation='sigmoid')(model.output)
            valence_output = Lambda(lambda x: x * 8 + 1, name='valence_output')(valence_output)
            
            arousal_output = Dense(1, activation='sigmoid')(model.output)
            arousal_output = Lambda(lambda x: x * 8 + 1, name='arousal_output')(arousal_output)

            # Combine the model
            final_model = Model(inputs=model.input, outputs=[valence_output, arousal_output])

            # Compile the model
            final_model.compile(optimizer='adam',
                                loss={'valence_output': 'mse',
                                    'arousal_output': 'mse'},
                                metrics=[tf.keras.metrics.RootMeanSquaredError()])

            return final_model

        # Hyperparameter search
        tuner = RandomSearch(
            model_builder,
            objective='val_loss',
            max_trials=5,
            executions_per_trial=3,
            directory='random_search',
            project_name='emotion_recognition'
        )

        early_stopping_callback = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

        tuner.search(X_train, {'valence_output': y_valence_train, 'arousal_output': y_arousal_train},
                    validation_split=0.2, epochs=1, batch_size=32,
                    callbacks=[early_stopping_callback])

        # Get the best model
        best_model = tuner.get_best_models(num_models=1)[0]

        # Train the best model
        history = best_model.fit(X_train, {'valence_output': y_valence_train, 'arousal_output': y_arousal_train},
                                validation_split=0.2, epochs=50, batch_size=32,
                                callbacks=[early_stopping_callback])

        evaluation = best_model.evaluate(X_test, [y_valence_test, y_arousal_test])

        # Save the evaluation results in the dictionary
        key = f"video_{video}_split_{split_index}"
        results[key] = {
            'valence_output_root_mean_squared_error': evaluation[3],
            'arousal_output_root_mean_squared_error': evaluation[4]
        }

    # Calculate the average of the evaluation results
    evaluation_sum = np.array([list(rmse_dict.values()) for rmse_dict in results.values()]).sum(axis=0)
    average_evaluation = evaluation_sum / len(splits)

    # Save the results for the current video
    video_results[f"video_{video}"] = results

    # Save the average evaluation in the dictionary
    video_results[f"video_{video}_average"] = {
        'valence_output_root_mean_squared_error': average_evaluation[0],
        'arousal_output_root_mean_squared_error': average_evaluation[1]
    }

    # Reset the results dictionary for the next video
    results = {}


In [ ]:
print("Results for each split:")
print(results)

In [ ]:
print(final_model.evaluate(X_test, [y_valence_test, y_arousal_test]))

In [ ]:
print("X_train shape:", X_train.shape)
print("y_valence_train shape:", y_valence_train.shape)
print("y_arousal_train shape:", y_arousal_train.shape)
print("X_test shape:", X_test.shape)
print("y_valence_test shape:", y_valence_test.shape)
print("y_arousal_test shape:", y_arousal_test.shape)

In [ ]:
[split for split_index, split in enumerate(splits)]

In [ ]:
X_test = np.swapaxes(X_test, 0, 1)

In [ ]:
print(final_model.evaluate(X_test, [y_valence_test, y_arousal_test]))


In [ ]:
print(X_test.shape)
print(y_valence_test.shape)
print(y_arousal_test.shape)
